In [1]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd

# 環境変数の取得
load_dotenv()

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [2]:
# 1. Excelファイルを読み込む
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='売上データ')
# データフレームを表示して確認
df.head()

,カテゴリー,商品コード,商品名,売上日,単価,数量,原価
0,食品,1001,りんご,2023-01-01,200,50,120
1,食品,1002,バナナ,2023-01-01,150,100,80
2,食品,1003,牛乳,2023-01-02,180,80,100
3,衣服,2001,Tシャツ,2023-01-02,1500,20,800
4,衣服,2002,ジーンズ,2023-01-03,5000,10,2500


In [3]:
# 2. データをLLM用にテキスト形式に変換
# データフレーム全体を文字列に変換
sales_data_text = df.astype(str)
prompt_text = f"売上データ:\n{sales_data_text}\nこの売上データの傾向を分析してください。"
# 表示して確認
print(prompt_text)

売上データ:
    カテゴリー 商品コード      商品名         売上日    単価   数量    原価
0      食品  1001      りんご  2023-01-01   200   50   120
1      食品  1002      バナナ  2023-01-01   150  100    80
2      食品  1003       牛乳  2023-01-02   180   80   100
3      衣服  2001     Tシャツ  2023-01-02  1500   20   800
4      衣服  2002     ジーンズ  2023-01-03  5000   10  2500
..    ...   ...      ...         ...   ...  ...   ...
235    衣服  2077   レインパンツ  2023-04-28  2000   18  1000
236    食品  1085      ザクロ  2023-04-29   600   40   300
237   日用品  3077    バスブラシ  2023-04-29   400   60   200
238    衣服  2078  レインシューズ  2023-04-30  2500   15  1250
239    食品  1086    ココナッツ  2023-04-30   300   80   150

[240 rows x 7 columns]
この売上データの傾向を分析してください。


In [8]:
# 3. OpenAI APIの呼び出し

# 役割を設定
role = "あなたはマーケティング分野に精通したデータサイエンティストです。企業の成長をサポートするために、効果的なインサイトを提供します。"

# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": role},
        {"role": "user", "content": prompt_text},
    ],
)

# LLMからの回答を表示
print(response.choices[0].message.content.strip())

与えられた売上データを分析するために、様々な視点からデータの傾向を掘り下げてみましょう。具体的には、売上のカテゴリー別傾向、時間的な傾向、利益率の分析、及び販売数量に焦点を当てます。

### 1. カテゴリー別の売上分析
各カテゴリーの売上、数量、原価、利益を算出し、どのカテゴリーが全体の成長に寄与しているかを確認します。

- **売上の合計**
  - 食品: 売上高、数量共に大きい
  - 衣服: 単価が高いため、売上高も大きい
  - 日用品: 売上は少なめだが、安定性が特徴

- **利益の計算**
  - 利益 = 売上 - 原価
  - 各カテゴリーごとに利益の合計を求めることで、最も利益をもたらしているカテゴリーを特定します。

### 2. 時間的な傾向
売上の日付別、または月別に集計し、シーズナリティを分析します。

- **月別売上のトレンド**
  - 特定の月に売上が増加しているか（例: 年末商戦、春のセールなど）
  - 基本的なトレンドを特定し、どの時期にマーケティングの強化が必要かを示唆します。

- **平均単価と数量**
  - 時間ごとの平均単価や数量を計算し、トレンドの変化を分析します。

### 3. 利益率分析
各商品の利益率を算出し、どの製品がより効率的に利益を生んでいるかを確認します。

- **利益率 = (売上 - 原価) / 売上**
  - 利益率が高い商品に焦点を当てて、製品戦略を見直すことが可能です。

### 4. 販売数量のランキング
最も売れた商品、もしくは売れ行きが良い商品をランキングし、その商品の特徴を分析します。

### 結論
これらの分析から、以下のインサイトを得ることができるでしょう。

- 利益を最大化するためには、どのカテゴリーと商品を中心にマーケティングや在庫管理を行うべきか明らかになります。
- 特定の時期に売上が増加する傾向がある場合、その時期に向けたプロモーション戦略を強化できます。
- 利益率が高い商品を中心に売上を拡大することで、総売上高の向上に貢献します。

この分析結果を基に、戦略的な意思決定ができるよう、さらなる具体的な施策やアクションプランを策定することが重要です。具体的なデータがあれば、より詳細な数値分析を行うことが可能となります。


In [9]:
# 4. 分析結果をデータフレームに変換
result_list = response.choices[0].message.content.strip().split("\n")
df_out = pd.DataFrame(result_list, columns=['結果'])
print(df_out)

                                                   結果
0   与えられた売上データを分析するために、様々な視点からデータの傾向を掘り下げてみましょう。具体...
1                                                    
2                                  ### 1. カテゴリー別の売上分析
3   各カテゴリーの売上、数量、原価、利益を算出し、どのカテゴリーが全体の成長に寄与しているかを確...
4                                                    
5                                         - **売上の合計**
6                                   - 食品: 売上高、数量共に大きい
7                               - 衣服: 単価が高いため、売上高も大きい
8                              - 日用品: 売上は少なめだが、安定性が特徴
9                                                    
10                                        - **利益の計算**
11                                     - 利益 = 売上 - 原価
12    - 各カテゴリーごとに利益の合計を求めることで、最も利益をもたらしているカテゴリーを特定...
13                                                   
14                                      ### 2. 時間的な傾向
15                    売上の日付別、または月別に集計し、シーズナリティを分析します。
16                                                   
17                          

In [10]:
# 5. 結果をExcelファイルに保存
df_out.to_excel("売上データ分析結果.xlsx", index=False)